In [1]:
import re, random
from os import path, makedirs
from urllib.error import HTTPError
from time import sleep
import pandas as pd
import numpy as np
from Bio import Entrez, SeqIO
from Bio.Alphabet import generic_dna

Entrez.email = # your email here
bed_file = "http://jaspar.genereg.net/download/bed_files/MA0852.2.bed"


In [3]:
def expand_query(query: pd.Series) -> pd.DataFrame:
    """ Parses query formatted for UCSC Biologic database """
    # expand query
    parser = re.compile(r"^([a-z]{2,4}\d{,2})(?:\_)(chr\d{,2}[A-Z]{,3})(?:[^\:]*):(\d+)-(\d+)(?:\()([\-\+])(?:\))")
    results = query.apply(parser.findall).apply(lambda r: list(sum(r, ()))).apply(pd.Series)
    return results

def load_jasper_bed_files(filename):
    """ loads jasper bed files which contain
        jasper motif-id and query of motif
    """
    final_cols = ["motif-id", "organism", "genome", "chromosome", "start", "stop", "strand"]
    # read csv
    locus_file = pd.read_csv(filename,
                             sep="\t",
                             header=None)

    # 3: query -> organism, chormosome, start, stop, strand
    result_df = expand_query(locus_file.iloc[:,3])

    # name columns
    result_df.columns = final_cols[2:]
    
    # convert start and stop
    result_df['start'] = result_df['start'].apply(int)
    result_df['stop'] = result_df['stop'].apply(int)

    # motif id is part of file name
    result_df["motif-id"] = filename[-12:-4]

    # switch org names to Latin
    result_df["organism"] = "homo sapiens"

    # reorder columns
    return result_df[final_cols]

In [4]:
bed_df = load_jasper_bed_files(bed_file)

In [5]:
bed_df.head()

,motif-id,organism,genome,chromosome,start,stop,strand
0,MA0852.2,homo sapiens,hg38,chr1,1304891,1304904,+
1,MA0852.2,homo sapiens,hg38,chr1,2311718,2311731,-
2,MA0852.2,homo sapiens,hg38,chr1,3403994,3404007,-
3,MA0852.2,homo sapiens,hg38,chr1,3624782,3624795,+
4,MA0852.2,homo sapiens,hg38,chr1,6601250,6601263,-


In [30]:
def id_to_refseq(org, chrom):
    id_switch = {
            'homo sapiens':
            lambda chrom:
            f"NC_0000{(int(chrom)):02d}" if
            chrom.isnumeric() and (0 < int(chrom) < 23) else
            "NC_000023" if chrom == "X" else
            "NC_000024" if chrom == "Y" else
            None,
    }
    return id_switch[org](chrom)

def fetch_chromosome(iden):
    """ fetch fasta sequence from NCBI given id """
    try:
        with Entrez.efetch(db='nucleotide', id=iden, rettype='fasta') as handle:
            rec = SeqIO.read(handle, 'fasta')
            rec.Alphabet = generic_dna
            return rec
    except HTTPError:
        return None

def download_needed_chromosomes(datf) -> int:
    """save chromosome sequences as fasta files in sequences folder
    returns: number of sequences successfully saved
    """
    saved = 0
    # for each chromosome:
    for org, chrom in datf[['organism', 'chromosome']] \
                           .groupby(['organism', 'chromosome']).count().index:

        # get id for looking in NCBI
        refseq_id = id_to_refseq(org, chrom.strip('chr'))

        outfilename = "tmp/" + org.replace(" ", "-") +\
                          chrom.strip('chr') + ".fasta"
            
        if not path.isdir("./tmp/"):
            makedirs("./tmp/")

        # fetch sequence from NCBI
        big_sequence = fetch_chromosome(refseq_id)

        if not big_sequence:
            print('error: in {} {} refseq_id {}'.format(org, chrom, refseq_id))

        else:
            # save sequence
            SeqIO.write(big_sequence, outfilename, 'fasta')

            # update status
            saved += 1
            print('saved', big_sequence.description)

            # ensure requests do not go to NCBI too fast
            sleep(1)
    return saved
    

In [31]:
download_needed_chromosomes(bed_df)

saved NC_000001.11 Homo sapiens chromosome 1, GRCh38.p12 Primary Assembly
saved NC_000010.11 Homo sapiens chromosome 10, GRCh38.p12 Primary Assembly
saved NC_000011.10 Homo sapiens chromosome 11, GRCh38.p12 Primary Assembly
saved NC_000012.12 Homo sapiens chromosome 12, GRCh38.p12 Primary Assembly
saved NC_000013.11 Homo sapiens chromosome 13, GRCh38.p12 Primary Assembly
saved NC_000014.9 Homo sapiens chromosome 14, GRCh38.p12 Primary Assembly
saved NC_000015.10 Homo sapiens chromosome 15, GRCh38.p12 Primary Assembly
saved NC_000016.10 Homo sapiens chromosome 16, GRCh38.p12 Primary Assembly
saved NC_000017.11 Homo sapiens chromosome 17, GRCh38.p12 Primary Assembly
saved NC_000018.10 Homo sapiens chromosome 18, GRCh38.p12 Primary Assembly
saved NC_000019.10 Homo sapiens chromosome 19, GRCh38.p12 Primary Assembly
saved NC_000002.12 Homo sapiens chromosome 2, GRCh38.p12 Primary Assembly
saved NC_000020.11 Homo sapiens chromosome 20, GRCh38.p12 Primary Assembly
saved NC_000021.9 Homo sapie

23

In [7]:
LENGTH = 1000
RAND_LIM = 500

def choose_row(r, chrom):
    if r["strand"] == "+":
        return str(chrom[r["rstart"]-1:r["rstop"]].seq)
    else: 
        return str(chrom[r["rstart"]-1:r["rstop"]].seq.reverse_complement())

def search_chromosome(chrom: SeqIO.SeqRecord, start: pd.Series,
                      stop: pd.Series, strand: pd.Series) -> tuple:
    """ returns (motif, mstart, mstop) """
    
    # length of motif
    len_motifs = (stop - start) + 1 # plus 1 because 0 vs. 1 indexing
    
    rstart = start - len_motifs.apply(lambda d: np.random.randint(0, LENGTH - d))
    rstop = rstart + LENGTH
    # get randomized indents within a set range

    # select motif +/- indents
    motifs = pd.concat([rstart, rstop, strand], keys=["rstart", "rstop", 'strand'], axis=1)
    motifs = motifs.apply(lambda r: choose_row(r, chrom), axis=1)

    # return motif, start index from selected sequence, and
    # stop index from selected sequence
    return (motifs, start - rstart, start - rstart + len_motifs)

def load_sequences(datf):
    """ load chromosomes and return dataframe of sequences with motifs """
    dirname = "tmp/"
    seqdat = pd.DataFrame()
    for org, chrom in datf[['organism', 'chromosome']] \
                           .groupby(['organism', 'chromosome']).count().index:

        chrom_file = dirname + org.replace(" ", "-") +\
                     chrom.strip("chr") + ".fasta"
        chrom_record = SeqIO.read(chrom_file, 'fasta')

        # get rows for organism and chromosome
        startstops = datf.loc[(datf['organism'] == org) & (datf['chromosome'] == chrom)]
        # retrive motif + indent
        motifs, mstarts, mstops = search_chromosome(chrom_record,
                                                    startstops["start"],
                                                    startstops["stop"],
                                                    startstops["strand"])
        rows = pd.concat([startstops, motifs, mstarts, mstops], axis=1)
        rows.columns = ["motif-id", "organism", "genome", "chromosome", "start",
                       "stop", "strand", "seq", "mstart", "mstop"]
        seqdat = seqdat.append(rows, ignore_index=True)

    return seqdat

In [8]:
seq_df = load_sequences(bed_df)

In [16]:
seq_df.head()

,motif-id,organism,genome,chromosome,start,stop,strand,seq,mstart,mstop
0,MA0852.2,homo sapiens,hg38,chr1,1304891,1304904,+,CCCTCACCCCACTGCACTACTGCAGCCCATCCAGGTCTGGCACCCA...,359,373
1,MA0852.2,homo sapiens,hg38,chr1,2311718,2311731,-,GTGGGGCACAGGGTGGGGCAGAAACTAGGGAGGTGGGGGAGCCCCC...,542,556
2,MA0852.2,homo sapiens,hg38,chr1,3403994,3404007,-,GGGGGTGTAGGGAGCACGGGGGCGGGCGCCCGTCTGCAGCCTCCTG...,24,38
3,MA0852.2,homo sapiens,hg38,chr1,3624782,3624795,+,GCACACACATGTTCCACCTCGAACTGCGAGCGCTGACCGGGATCTC...,482,496
4,MA0852.2,homo sapiens,hg38,chr1,6601250,6601263,-,CCCGGAGCCTGTAAACATGGTCACATGTTAAGAGGTTTTCTTCCAG...,980,994


In [17]:
seq_df.to_csv('./MA0852_2/MA0852.2SEQS1k.csv', sep=",", header=True, index=False)